# Assignment 7: Automated Machine Learning (Part 2)
## Objective:

As we learned from the class, the high demand for machine learning has produced a large amount of data scientists who have developed expertise in tools and algorithms. The features in the data will directly influence the results. However, it is tedious and unscalable to manually design and select features without domain knowledge. Thus, using some AutoML techniques will significantly help data scientists save labour and time. 
After completing this assignment, you should be able to answer the following questions:

1. Why do we need AutoML?
2. How does auto feature generation work?
3. How to use featuretools library to automatically generate features?
4. How to get useful features in a large feature space?

Imagine you are a data scientist in an online retailer company, for example, Amazon. Your task is to provide some recommendations to customers based on their historical purchase records.

In this assignment, we predict whether the customer will buy **Banana** in the next 4 weeks. It is a classification problem. To simplify the problem, we have already generated some features and provide the accuracy of the model (Random Forest model). The task for you is to generate **10** useful features and beat our model performance (AUC = 0.61, see below). 

For example, <br>
`MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas` means whether the most frequent purchase of the customer is Bag of Organic Bananas. 

```
1: Feature: MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas
2: Feature: MODE(order_products.aisle_id) is unknown
3: Feature: SUM(orders.NUM_UNIQUE(order_products.product_name))
4: Feature: MODE(orders.MODE(order_products.product_name)) = Boneless Skinless Chicken Breasts
5: Feature: MODE(order_products.product_name) = Boneless Skinless Chicken Breasts
6: Feature: STD(orders.NUM_UNIQUE(order_products.aisle_id))
7: Feature: MODE(order_products.aisle_id) = 83
8: Feature: MEDIAN(orders.MINUTE(order_time))
9: Feature: MODE(orders.DAY(order_time)) = 23
10: Feature: MODE(orders.MODE(order_products.department)) = produce

AUC 0.61
```


## Preliminary
If you never use featuretools before, you need to learn some basic knowledge of this topic. 
I found that these are some good resources: 
* [featuretools documentation](https://docs.featuretools.com/en/stable/)
* [Tutorial: Automated Feature Engineering in Python](https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219)

The data can be downloaded from [A7-2-data.zip](A7-2-data.zip). 

## 0. Preparation
Import relevant libraries and load the dataset: <br>
users: <br>
* user_id: customer identifier
* label:  1 if the customer will buy banana in next 4 weeks, 0 otherwise

orders: <br>
* order_id: order identifier
* user_id: customer identifier
* order_time: date of the order was placed on 

order_products: <br>
* order_id: order identifier
* order_product_id: foreign key
* reordered:  1 if this product has been ordered by this user in the past, 0 otherwise
* product_name: name of the product
* aisle_id: aisle identifier
* department: the name of the department
* order_time: date of the order was placed on

In [70]:
import pandas as pd

import featuretools as ft
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import os
ft.__version__

'0.13.3'

In [71]:
orders = pd.read_csv("orders.csv",index_col=0)
order_products = pd.read_csv("order_products.csv",index_col=0)
users = pd.read_csv("users.csv",index_col=0)

print(users["label"].value_counts())
orders.shape

False    628
True     139
Name: label, dtype: int64


(5997, 3)

## Task 1. Feature Generation
In this task, you need to use featuretools to generate candidate features by using the above three tables.

### 1.1 Representing Data with EntitySet

Define entities and their relationships (see [https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html](https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html))

In [72]:
def load_entityset(orders, order_products, users):
    # create a entityset 
    es = ft.EntitySet(id = 'clients')
    # read three df as entity
    es = es.entity_from_dataframe(entity_id = 'users', dataframe = users, index = 'user_id')
    es = es.entity_from_dataframe(entity_id = 'orders', dataframe = orders, index = 'order_id', time_index='order_time' )
    es = es.entity_from_dataframe(entity_id = 'order_products', dataframe = order_products, index = 'order_product_id', time_index='order_time')
    # add realationships
    r_users_orders = ft.Relationship(es['users']['user_id'],es['orders']['user_id'])
    es = es.add_relationship(r_users_orders)
    r_orders_order_products = ft.Relationship(es['orders']['order_id'],es['order_products']['order_id'])
    es = es.add_relationship(r_orders_order_products)
    return es

### 1.2 Deep Feature Synthesis

In [73]:
# Automatically generate features
es = load_entityset(orders, order_products, users)

# use ft.dfs to perform feature engineering
# --- Write your code below ---
feature_matrix, feature_defs = ft.dfs(entityset = es, target_entity = 'users', 
                                 agg_primitives = ['mean', 'max', 'avg_time_between', 'time_since_last', 'percent_true', 'std', 'sum', 'median'])

feature_matrix_encoding, features_encoding = ft.encode_features(feature_matrix, feature_defs)
feature_matrix = feature_matrix_encoding
feature_matrix = feature_matrix.fillna(0)



In [74]:
# output what features you generate
feature_matrix

,label,AVG_TIME_BETWEEN(orders.order_time),TIME_SINCE_LAST(orders.order_time),MEAN(order_products.aisle_id),MEAN(order_products.reordered),MAX(order_products.aisle_id),MAX(order_products.reordered),AVG_TIME_BETWEEN(order_products.order_time),TIME_SINCE_LAST(order_products.order_time),STD(order_products.aisle_id),...,MEDIAN(orders.STD(order_products.reordered)),MEDIAN(orders.MEDIAN(order_products.aisle_id)),MEDIAN(orders.MAX(order_products.reordered)),MEDIAN(orders.TIME_SINCE_LAST(order_products.order_time)),MEDIAN(orders.MEDIAN(order_products.reordered)),MEDIAN(orders.MEAN(order_products.aisle_id)),MEDIAN(orders.SUM(order_products.aisle_id)),MEDIAN(orders.SUM(order_products.reordered)),AVG_TIME_BETWEEN(order_products.orders.order_time),TIME_SINCE_LAST(order_products.orders.order_time)
user_id,,,,,,,,,,,,,,,,,,,,,
1,False,1903200.0,1.573825e+08,60.523810,0.523810,121,1,285480.000000,1.573825e+08,38.070486,...,0.273861,54.00,1.0,1.608421e+08,0.75,61.283333,309.0,3.0,285480.000000,1.573825e+08
2,True,843600.0,1.580197e+08,63.752941,0.341176,123,1,60257.142857,1.580197e+08,39.596448,...,0.408248,48.00,1.0,1.611589e+08,0.00,58.769231,764.0,1.0,60257.142857,1.580197e+08
3,False,1402200.0,1.574617e+08,69.048780,0.341463,123,1,140220.000000,1.574617e+08,40.594305,...,0.500000,78.00,1.0,1.603525e+08,0.00,68.000000,651.0,3.0,140220.000000,1.574617e+08
7,False,2043600.0,1.569577e+08,65.493151,0.493151,123,1,85150.000000,1.569577e+08,35.379342,...,0.391534,66.25,1.0,1.590997e+08,0.50,67.187500,1203.5,8.0,85150.000000,1.569577e+08
10,False,1671600.0,1.580557e+08,67.342105,0.175439,123,1,44378.761062,1.580557e+08,37.683787,...,0.285147,60.00,1.0,1.598719e+08,0.00,60.954545,1921.5,3.5,44378.761062,1.580557e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,False,1437600.0,1.587577e+08,52.711111,0.400000,128,1,98018.181818,1.587577e+08,44.964947,...,0.477463,55.50,1.0,1.617925e+08,0.25,56.866071,610.5,4.0,98018.181818,1.587577e+08
997,False,1833600.0,1.575661e+08,64.272727,0.212121,129,1,171900.000000,1.575661e+08,40.314601,...,0.240192,59.50,0.5,1.601365e+08,0.00,70.330769,545.0,1.5,171900.000000,1.575661e+08
998,False,984600.0,1.571701e+08,64.966667,0.683333,116,1,100128.813559,1.571701e+08,37.087147,...,0.000000,68.50,1.0,1.588153e+08,1.00,66.857143,478.0,5.0,100128.813559,1.571701e+08


## Task 2. Feature Selection
In this task, you are going to select 10 features that are useful and train the *Random Forest* model. The goal is to beat the accuracy performance as we have shown before. Note that you have to use the Random Forest and the hyperparameters we provide in Section 2.2. In other words, your job is to achieve a higher AUC than 0.61 through feature generation/selection rather than through hyperparameter tuning or model selectoin. 

### 2.1 Select top features

In [75]:
# --- Write your code below ---
# Select top-10 features and return X, y (X.shape = (767, 10)
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
y = feature_matrix['label']
X = feature_matrix.drop(columns=['label'])
columns=X.columns
# Removing features with low variance
sel = VarianceThreshold(threshold=(.3 * (1 - .3)))
matrix=sel.fit_transform(X)
# Univariate feature selection
selector=SelectKBest(chi2,k=10)
X = selector.fit_transform(matrix, y)
selector.fit(matrix,y)
cols=selector.get_support(indices=True)
columns=[columns[i] for i in cols]

### 2.2 Get accuracy and list features

In [76]:
clf = RandomForestClassifier(n_estimators=400, n_jobs=-1)
scores = cross_val_score(estimator=clf,X=X, y=y, cv=3,
                             scoring="roc_auc", verbose=True)

print("AUC %.2f" % (scores.mean()))

# Print top-10 features
for col in columns:
    print(col)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


AUC 0.66
AVG_TIME_BETWEEN(orders.order_time)
MAX(order_products.aisle_id)
TIME_SINCE_LAST(order_products.order_time)
MEAN(orders.AVG_TIME_BETWEEN(order_products.order_time))
MEAN(orders.TIME_SINCE_LAST(order_products.order_time))
MEAN(orders.SUM(order_products.reordered))
MAX(orders.SUM(order_products.reordered))
STD(orders.TIME_SINCE_LAST(order_products.order_time))
STD(orders.MEAN(order_products.aisle_id))
STD(orders.SUM(order_products.reordered))


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.5s finished


## Task3. Writing Questions

1. Please list three advantages and disadvantages of featuretools. 
Advantanges:
a.We can stack a deep feature made of stacking multiple primitives
b.We can stack features to any depth we want.
c.We do not have to manually specify the feature primitives, but instead can let featuretools automatically choose features for us.
Disadvantages:
a.Automated feature engineering has solved one problem, but created another: too many features. Although itâ€™s difficult to say before fitting a model which of these features will be important, itâ€™s likely not all of them will be relevant to a task we want to train our model on. 
b.Moreover, having too many features can lead to poor model performance because the less useful features drown out those that are more important.
c.The problem of too many features is known as the curse of dimensionality.
2. For those disadvantages you listed above, do you have any idea to improve it? 
The curse of dimensionality is combated with feature reduction (also known as feature selection): the process of removing irrelevant features. This can take on many forms: Principal Component Analysis (PCA), SelectKBest, using feature importances from a model, or auto-encoding using deep neural networks. 

--- Write your answer here---

## Submission
Complete the code in this notebook, and submit it to the CourSys activity Assignment 7.